In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import cProfile
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.cross_validation import train_test_split
from scipy.spatial.distance import cdist
from kmeansplusplus import mykmeansplusplus

In [2]:
def dataset_fixed_cov():
    '''Generate 2 Gaussians samples with the same covariance matrix'''
    n, dim = 10000000, 2
    np.random.seed(0)
    #C = np.array([[0., -0.23], [0.83, .23]])
    C = np.array([[0.,-0.23],[0.5,0.23]])
    X = np.r_[np.dot(np.random.randn(n, dim), C),
              np.dot(np.random.randn(n, dim), C) + np.array([1, 1])]
    y = np.hstack((np.zeros(n), np.ones(n)))
    return X, y

X,y = dataset_fixed_cov()
random_num = np.random.randint(X.shape[0], size = X.shape[0])
random_number = 11232015
rng = np.random.RandomState(random_number)
permutation1 = rng.permutation(len(random_num))
random_num = random_num[permutation1]
X = X[random_num]
y = y[random_num]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=11)
X_train = preprocessing.scale(X_train,axis = 0, with_mean=True, with_std=True)

In [3]:
def f(C,x):
    Y= cdist(x, C, metric='euclidean', p=2, V=None, VI=None, w=None)
    target_dist = np.amin(Y, axis=1)
    
def online_k_means(k,b,t,X_in):
    random_number = 11232015
    random_num = np.random.randint(X_in.shape[0], size =300 )
    rng = np.random.RandomState(random_number)
    permutation1 = rng.permutation(len(random_num))
    random_num = random_num[permutation1]
    x_input = X_in[random_num]
    c,l = mykmeansplusplus(x_input,k,t)
    v = np.zeros((k))
    for i in range(t):
        random_num = np.random.randint(X_in.shape[0], size = b)
        rng = np.random.RandomState(random_number)
        permutation1 = rng.permutation(len(random_num))
        random_num = random_num[permutation1]
        M = X_in[random_num]
        Y = cdist(M,c,metric='euclidean', p=2, V=None, VI=None, w=None)
        clust_index = np.argmin(Y,axis = 1)
        for i in range(M.shape[0]):
            c_in = clust_index[i]
            v[c_in] += 1
            ita = 1 / v[c_in]
            c[c_in] = np.add(np.multiply((1 - ita),c[c_in]),np.multiply(ita,M[i]))
    Y_l = cdist(X_in,c,metric='euclidean', p=2, V=None, VI=None, w=None)
    l = np.argmin(Y_l,axis = 1)        
    return c,l

In [ ]:
def my_kmeans_new(k,t,x,b):
    if x.shape[0] > 1000:
        return online_k_means(k,b,t,x)
    else:
        return mykmeansplusplus(x,k,t)

In [4]:
def compare_k_means(k,t,x):
    b = 100
    start_time = time.time()
    c1,l1 = online_k_means(k,b,t,x)
    print("--- %s seconds for online---" % (time.time() - start_time))
    start_time = time.time()
    c2,l2 = mykmeansplusplus(x,k,t)
    print("--- %s seconds for mykmean++---" % (time.time() - start_time))

    
    
    

In [5]:
#compare_k_means(30,1000,X_train)


9
converged
--- 2.95606303215 seconds for online---


MemoryError: 

In [ ]:
"""
centers,l = online_k_means(10,5,10,X_train)
print centers
print l

#from sklearn.metrics.pairwise import euclidean_distances
"""

In [ ]:
"""
initial_clusters,labels = mykmeansplusplus(X_train,10,100)
from sklearn.cluster import KMeans
k_means = KMeans(init='k-means++', n_clusters=10, n_init=100)
k_means.fit(X_train)
std_centers = k_means.cluster_centers_
#print initial_clusters
#print std_centers
#Y= cdist(initial_clusters, std_centers, metric='euclidean', p=2, V=None, VI=None, w=None)
#print Y
"""